# GEMM on GPU

## 1. Set-up

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [3]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

# %mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

/content/gdrive/MyDrive/ece5545
fatal: destination path 'a3-MaximClouser' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a3-MaximClouser
Already on 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), 427 bytes | 1024 bytes/s, done.
From https://github.com/ML-HW-SYS/a3-MaximClouser
   93999bf..2e6b1f2  main       -> origin/main
Updating 93999bf..2e6b1f2
Fast-forward
 src/ops.py | 9 +++++----
 1 file changed, 5 insertions(+), 4 deletions(-)
/content/gdrive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [5]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb  3-conv1d_fpga.ipynb  5-conv2d_dw_gpu.ipynb	README.md  tests
2-conv1d_gpu.ipynb  4-gemm_gpu.ipynb	 leaderboard_id.txt	src


## 2. Install TVM

In [6]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.6/428.6 MB 3.6 MB/s eta 0:00:00


## 3. Implement `make_conv1d_gpu_scheduler_func` function in `src.ops`

In that function, you are required to implemented 1D convolution and use TVM to optimize it.
Let $x \in \mathbb{R}^m$ and $y \in \mathbb{R}^n$, then
$$
\operatorname{conv1d}(x, y)_i = \sum_{j=-\infty}^{\infty} x[j]y[i-j], \forall i \in \{0, 1, \dots, m + n - 1\}
$$

Please use zero padding and unit stride. Please see the numpy convolution function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

The `make_conv1d_gpu_scheduler_func` takes $m$ and $n$, which are the size of the two 1D input array.
You should return both the TVM scheduler and the TVM opterator for
1. Input $x$
2. Input $y$
3. Output $out$

The scheduler should be able to used to build a function with signature $func(x, y, out)$.
Please see the following cells for usage.

In [85]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), 388 bytes | 8.00 KiB/s, done.
From https://github.com/ML-HW-SYS/a3-MaximClouser
   0b67da0..106279f  main       -> origin/main
Updating 0b67da0..106279f
Fast-forward
 src/ops.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [87]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_gemm_gpu_scheduler

M = 1024
N = 512
K = 2048
dtype = 'float32'
a_np = np.random.rand(M, K).astype(dtype)
w_np = np.random.rand(K, N).astype(dtype)
b_np = np.matmul(a_np, w_np)

s, A, W, B = make_gemm_gpu_scheduler(M, K, N)
func = tvm.build(s, [A, W, B], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M, N), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=50, repeat=5)


print("Answer:", b_np)
print("Output:", b)
print(f"GEMM TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [[536.87805 525.75806 513.7951  ... 515.15796 517.0174  526.30817]
 [530.047   525.05066 508.7304  ... 505.29764 509.09314 514.0283 ]
 [526.38184 515.00134 508.64276 ... 505.66125 507.46445 513.1376 ]
 ...
 [525.5017  521.48975 504.25616 ... 506.93036 504.70734 505.7844 ]
 [538.97363 523.1776  512.6109  ... 512.75226 515.8224  519.037  ]
 [516.27625 511.37625 499.4471  ... 486.71268 491.43408 499.63498]]
Output: [[536.87885 525.7578  513.7951  ... 515.15796 517.0175  526.30804]
 [530.047   525.05054 508.73083 ... 505.29813 509.09308 514.02826]
 [526.3817  515.0016  508.64297 ... 505.66113 507.46466 513.13794]
 ...
 [525.5016  521.4897  504.25632 ... 506.93088 504.70773 505.7847 ]
 [538.97327 523.1777  512.611   ... 512.7526  515.82263 519.03723]
 [516.2765  511.37607 499.4473  ... 486.71262 491.43375 499.63483]]
GEMM TVM: 12.563058 ms


In [ ]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

In [89]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_gemm_gpu.py

/content/gdrive/MyDrive/ece5545/a3-MaximClouser
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-MaximClouser
plugins: anyio-3.7.1
collected 20 items                                                                                 

tests/test_gemm_gpu.py ....................                                                  [100%]

================================== 20 passed in 182.54s (0:03:02) ==================================
